In [ ]:
import os
import time
import torch
import numpy as np
import pandas as pd
from torch import nn
from torchmetrics.regression import SymmetricMeanAbsolutePercentageError

import data
import models
import dataloader_last as dataloader
import inference
import training

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

# configure

In [1]:
#all_roads = ['nfb0019','nfb0425','nfb0431','nfb0033','nfb0061','nfb0064','nfb0327','nfb0328', 'nfb0063','nfb0247','nfb0248','nfb0117','nfb0124','nfb0123']
all_roads = ['nfb0019']

In [ ]:
root_path = "./nfb_final"
n_batch = 128
n_epoche = 10
n_workers = 0

# Training & Testing

In [ ]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
device

In [ ]:
dir_name = './best_model/'

In [ ]:
if not os.path.isdir(dir_name):
    os.mkdir(dir_name)

## Model

In [ ]:
model_name = 'PTE'
print(model_name)

In [ ]:
if not os.path.isdir(f'{dir_name}/{model_name}/'):
    os.mkdir(f'{dir_name}/{model_name}/')

In [ ]:
for road in all_roads:

    print(f'====={road}=====')

    path = f'{dir_name}/{model_name}/{road}/'
    if not os.path.isdir(path):
        os.mkdir(path)

    origin_df, trav_mean, trav_std = data.roads(root_path, road)

    for predict_time in range(0, 8):

        model_4 = models.DAT()
        model_4 = model_4.to(device)

        optimizer = torch.optim.AdamW(model_4.parameters(), lr=1e-4)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

        print('For predict time %s day' % predict_time)
        selecteddata = f'_{predict_time}d_last1h'
        loaders = data.create_dataloader(road, root_path, selecteddata, n_batch, n_workers)

        print('training...')
        training.transformer_train(n_epoche, model_4, loaders['train'], loaders['test'], trav_mean, trav_std, optimizer, scheduler, path, predict_time)

## Test

In [ ]:
road_all_mae, road_all_rmse, road_all_smape = {}, {}, {}
road_peak_mae, road_peak_rmse, road_peak_smape = {}, {}, {}
road_off_peak_mae, road_off_peak_rmse, road_off_peak_smape = {}, {}, {}


for road in all_roads:
    path = f'{dir_name}/{model_name}/{road}/'
    print('testing...')

    all_mae, all_rmse, all_smape = [], [], []
    peak_mae, peak_rmse, peak_smape = [], [], []
    off_peak_mae, off_peak_rmse, off_peak_smape = [], [], []

    origin_df, trav_mean, trav_std = data.roads(root_path, road)

    for predict_time in range(0, 8):
        print('testing...')

        model_4 = models.PTE()
        model_4 = model_4.to(device)
        model_4.load_state_dict(torch.load(path + f'model_{predict_time}.pth'))

        print('For predict time %s day' % predict_time)
        selecteddata = f'_{predict_time}d_last1h'
        loaders = data.create_dataloader(road, root_path, selecteddata, n_batch, n_workers)

        all_mae_loss, all_rmse_loss, all_smape_loss, \
        peak_mae_loss, peak_rmse_loss, peak_smape_loss, \
        off_peak_mae_loss, off_peak_rmse_loss, off_peak_smape_loss = inference.transformer_test_peak_off_peak(model_4, loaders['test'], trav_mean, trav_std)

        all_rmse.append(all_rmse_loss)
        all_mae.append(all_mae_loss)
        all_smape.append(all_smape_loss)

        peak_rmse.append(peak_rmse_loss)
        peak_mae.append(peak_mae_loss)
        peak_smape.append(peak_smape_loss)

        off_peak_rmse.append(off_peak_rmse_loss)
        off_peak_mae.append(off_peak_mae_loss)
        off_peak_smape.append(off_peak_smape_loss)

    road_all_rmse[road] = all_rmse
    road_all_mae[road] = all_mae
    road_all_smape[road] = all_smape

    road_peak_rmse[road] = peak_rmse
    road_peak_mae[road] = peak_mae
    road_peak_smape[road] = peak_smape

    road_off_peak_rmse[road] = off_peak_rmse
    road_off_peak_mae[road] = off_peak_mae
    road_off_peak_smape[road] = off_peak_smape

In [ ]:
pd.DataFrame(road_all_mae).T

In [ ]:
pd.DataFrame(road_all_rmse).T

In [ ]:
pd.DataFrame(road_all_smape).T

In [ ]:
pd.DataFrame(road_peak_mae).T

In [ ]:
pd.DataFrame(road_peak_rmse).T

In [ ]:
pd.DataFrame(road_peak_smape).T

In [ ]:
pd.DataFrame(road_off_peak_mae).T

In [ ]:
pd.DataFrame(road_off_peak_rmse).T

In [ ]:
pd.DataFrame(road_off_peak_smape).T

In [ ]:
final, final_peak, final_off_peak = {}, {}, {}
for road in roads:
    temp, temp_peak, temp_off_peak = [], [], []
    for i in range(8):
        temp.append(road_all_mae[road][i])
        temp.append(road_all_rmse[road][i])
        temp.append(road_all_smape[road][i])
        temp_peak.append(road_peak_mae[road][i])
        temp_peak.append(road_peak_rmse[road][i])
        temp_peak.append(road_peak_smape[road][i])
        temp_off_peak.append(road_off_peak_mae[road][i])
        temp_off_peak.append(road_off_peak_rmse[road][i])
        temp_off_peak.append(road_off_peak_smape[road][i])
    final[road] = temp
    final_peak[road] = temp_peak
    final_off_peak[road] = temp_off_peak

In [ ]:
pd.DataFrame(final).T.to_csv('trans all.csv')

In [ ]:
pd.DataFrame(final_peak).T.to_csv('trans peak.csv')

In [ ]:
pd.DataFrame(final_off_peak).T.to_csv('trans off.csv')